# Rakamin X Home Credit Indonesia End-to-end Solution for Credit Risk Analysis

- Name : Mohammad Nabil Syakir
- LinkedIn : https://www.linkedin.com/in/mohammadnabilsyakir/
- Github : https://github.com/nabilsyakir

# Introduction

__Problem__: We are investors in P2P platforms. There are loans that get charged off in the end. If a loan get charged off or defaulted, we will lost our money. We want to prevent that, and minimize our loss.

__Business Metrics__: Loss, net profit margin.

__Solution explanation__: We will create a machine learning model that can identify if a loan is potentially bad / risky loan. It can be used as an investment decision tools. For the model, we're gonna use some non-parametrical algorithm (with little assumption) because we are not statistician, and statistics is hard. If our model is reliable, our investment in risky loans will decrease, our loss can be minimized and our net profit margin should increase.

__Data__: Lending club credit loan data between 2007 - 2014

# Import Libraries

In [1]:
#import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import decimal

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
decimals = 0
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 500)

# Load Dataset

In [3]:
df = pd.read_csv('application_train.csv')

# Exploring Data

In [4]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [5]:
df.shape

(307511, 122)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [7]:
df.dtypes

SK_ID_CURR                        int64
TARGET                            int64
NAME_CONTRACT_TYPE               object
CODE_GENDER                      object
FLAG_OWN_CAR                     object
FLAG_OWN_REALTY                  object
CNT_CHILDREN                      int64
AMT_INCOME_TOTAL                float64
AMT_CREDIT                      float64
AMT_ANNUITY                     float64
AMT_GOODS_PRICE                 float64
NAME_TYPE_SUITE                  object
NAME_INCOME_TYPE                 object
NAME_EDUCATION_TYPE              object
NAME_FAMILY_STATUS               object
NAME_HOUSING_TYPE                object
REGION_POPULATION_RELATIVE      float64
DAYS_BIRTH                        int64
DAYS_EMPLOYED                     int64
DAYS_REGISTRATION               float64
DAYS_ID_PUBLISH                   int64
OWN_CAR_AGE                     float64
FLAG_MOBIL                        int64
FLAG_EMP_PHONE                    int64
FLAG_WORK_PHONE                   int64


## Statistical Summary

In [8]:
numerical_feature = df.select_dtypes(exclude='object')
categorical_feature = df.select_dtypes(include='object')

In [9]:
numerical_feature.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,104582.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307509.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,134133.000000,3.068510e+05,246546.000000,151450.00000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,151450.000000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,151450.000000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,159080.000000,306490.000000,306490.000000,306490.000000,306490.000000,307510.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.00000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,-4986.120328,-2994.202373,12.061091,0.999997,0.819889,0.199368,0.998133,0.281066,0.056720,2.152665,2.052463,2.031521,12.063419,0.015144,0.050769,0.040659,0.078173,0.230454,0.179555,0.502130,5.143927e-01,0.510853,0.11744,0.088442,0.977735,0.752471,0.044621,0.078942,0.149725,0.226282,0.231894,0.066333,0.100775,0.107399,0.008809,0.028358,0.114231,0.087543,0.977065,0.759637,0.042553,0.074490,0.145193,0.222315,0.228058,0.064958,0.105645,0.105975,0.008076,0.027022,0.117850,0.087955,0.97

In [10]:
categorical_feature.describe()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE
count,307511,307511,307511,307511,306219,307511,307511,307511,307511,211120,307511,307511,97216,153214,151170,161756
unique,2,3,2,2,7,8,5,6,6,18,7,58,4,3,7,2
top,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Laborers,TUESDAY,Business Entity Type 3,reg oper account,block of flats,Panel,No
freq,278232,202448,202924,213312,248526,158774,218391,196432,272868,55186,53901,67992,73830,150503,66040,159428


# Exploratory Data Analysis

## Heatmap Correlation

In [ ]:
plt.figure(figsize=(70, 70))
sns.heatmap(df.corr(), annot=True, fmt='.2f')

<AxesSubplot:>

## Distribution of Target

In [ ]:
df['TARGET'].value_counts(normalize=True)*100

In [ ]:
plt.figure(figsize=(16,8))

colors = sns.color_palette("tab10", 7)
labels ="0 (good)", "1 (risky)"

df["TARGET"].value_counts().plot.pie(autopct='%1.2f%%', shadow=True,labels = labels ,colors=colors, fontsize=12, startangle=70)

plt.text(x=-1.4, y=1.4, s="8.07% of the total borrowers have the Potential \nto Default", 
         fontsize=20, fontweight='bold')
plt.text(x=-1.4, y=1.15, s="The '1' Label explain Client with payment difficulties \nex: late payment", 
         fontsize=15)

## Application Every Year

In [ ]:
df['DATE_REGISTRATION'] = pd.to_datetime("today") + pd.to_timedelta(df['DAYS_REGISTRATION'], unit = 'D')
df['YEAR_REGISTRATION'] = pd.DatetimeIndex(df['DATE_REGISTRATION']).year
df['MONTH_REGISTRATION'] = pd.DatetimeIndex(df['DATE_REGISTRATION']).month
df['DAY_REGISTRATION']= pd.DatetimeIndex(df['DATE_REGISTRATION']).day

In [ ]:
df = df.drop(['DATE_REGISTRATION'], axis = 1)

In [ ]:
print(f'''(Year) Max Application''')
print(df['YEAR_REGISTRATION'].max())
print('')
print(f'''Year Value Counts''')
print(df['YEAR_REGISTRATION'].value_counts())

In [ ]:
year_regis_grouped = df.groupby(['YEAR_REGISTRATION']).agg({'AMT_CREDIT' : ['mean']})
year_regis_grouped = year_regis_grouped.reset_index()
year_regis_grouped

In [ ]:
plt.figure(figsize=(40,20))
sns.barplot(x='YEAR_REGISTRATION',y='AMT_CREDIT', data=df)

plt.text(x= -1, y=1609999, s="Generally, the nominal loan lending to customers is stagnant", 
         fontsize=42, fontweight='bold')
plt.text(x= -1, y=1500799, s="The nominal credit chart has tended to stagnate since 1978 and is expected to do so until 2021. \nIt appears to have increased by 2022. The company can investigate this further.", 
         fontsize=38)

plt.xlabel('Year', fontsize=14)
plt.ylabel('Average Credit Amount per Year', fontsize=14)

## Age Category VS Target

In [ ]:
df['AGE'] = round(df['DAYS_BIRTH']/-365)
df['AGE']

In [ ]:
df.loc[(df['AGE'] >= 0) & (df['AGE'] < 20), 'AGE_CATEGORY'] = "Late Ten"
df.loc[(df['AGE'] >= 20) & (df['AGE'] < 30), 'AGE_CATEGORY'] = "Twenty"
df.loc[(df['AGE'] >= 30) & (df['AGE'] < 40), 'AGE_CATEGORY'] = "Thirty"
df.loc[(df['AGE'] >= 40) & (df['AGE'] < 50), 'AGE_CATEGORY'] = "Fourty"
df.loc[(df['AGE'] >= 50) & (df['AGE'] < 60), 'AGE_CATEGORY'] = "Fifty"
df.loc[(df['AGE'] >= 60), 'AGE_CATEGORY'] = "Above_Sixty"

In [ ]:
print(f'''Min value Age Category Column:''')
print(df['AGE_CATEGORY'].min())

print(f'''Max value Age Category Column:''')
print(df['AGE_CATEGORY'].max())

print(f'''Value count Age Column:''')
print(df['AGE_CATEGORY'].value_counts())

In [ ]:
df_age = df.groupby(['AGE_CATEGORY','TARGET']).agg({'SK_ID_CURR' : ['nunique']})
df_age = df_age.reset_index()
df_age.columns = ['AGE CATEGORY','TARGET STATUS','TOTAL LOANERS']
df_age['RATIO'] = (df_age['TOTAL LOANERS'])/len(df)*100
df_age['RATIO ROUNDED'] = df_age['RATIO'].apply(lambda x: round(x, 1))
df_age

In [ ]:

sns.displot(x='AGE_CATEGORY', hue='TARGET', data=df, stat = 'percent',height=5, aspect=1.5)

plt.text(x= -1, y=28, s="Around 2.6% Loaners in thirty is more likely to default then others", 
         fontsize=15, fontweight='bold')
plt.text(x=-1, y=25, s="In the graphic image below, Every Age Category is have\nPotential to default", 
         fontsize=13)

plt.xlabel('Age Category', fontsize=14)
plt.ylabel('(%) Total Loaners', fontsize=14)

## Income Category VS Target

In [ ]:
df['AMT_INCOME_TOTAL'].head()

In [ ]:
df['AMT_INCOME_TOTAL'].describe()

In [ ]:
Q1 = df['AMT_INCOME_TOTAL'].quantile(0.25)
Q2 = df['AMT_INCOME_TOTAL'].quantile(0.5)
Q3 = df['AMT_INCOME_TOTAL'].quantile(0.75)

In [ ]:
def income_category(income):
    if income <= Q1:
        return 'Low'
    elif income <= Q2:
        return 'Middle Low'
    elif income <= Q3:
        return 'Middle Up'
    else:
        return 'High'

df['INCOME_CATEGORY'] = df['AMT_INCOME_TOTAL'].apply(income_category)

In [ ]:
df['INCOME_CATEGORY'].value_counts()

In [ ]:
df_income = df.groupby(['INCOME_CATEGORY','TARGET']).agg({'SK_ID_CURR' : ['nunique']})
df_income = df_income.reset_index()
df_income.columns = ['INCOME CATEGORY','TARGET STATUS','TOTAL LOANERS']
df_income['RATIO'] = (df_income['TOTAL LOANERS'])/len(df)*100
df_income['RATIO ROUNDED'] = df_income['RATIO'].apply(lambda x: round(x, decimals))
df_income

In [ ]:
plt.figure(figsize=(20,12))
sns.displot(x='INCOME_CATEGORY', hue='TARGET', data=df, stat = 'percent',height=5, aspect=1.5)

plt.text(x= -1, y=40, s="Around 3% Loaners in Low Income Category is more likely to default", 
         fontsize=15, fontweight='bold')
plt.text(x=-1, y=35, s="In the graphic image below, Every Income Category is have\nPotential to default", 
         fontsize=13)

plt.xlabel('Income Category', fontsize=14)
plt.ylabel('(%) Total Loaners', fontsize=14)

## Job Type VS Target

In [ ]:
df['NAME_INCOME_TYPE'].value_counts()

In [ ]:
df_job = df.groupby(['NAME_INCOME_TYPE','TARGET']).agg({'SK_ID_CURR' : ['nunique']})
df_job = df_job.reset_index()
df_job.columns = ['JOB CATEGORY','TARGET STATUS','TOTAL LOANERS']
df_job['RATIO'] = (df_job['TOTAL LOANERS'])/len(df)*100
df_job['RATIO ROUNDED'] = df_job['RATIO'].apply(lambda x: round(x, decimals))
df_job

In [ ]:
sns.displot(x='NAME_INCOME_TYPE', hue='TARGET', data=df, stat = 'percent', height=5, aspect=3)

plt.text(x= -1, y=58, s="Around 5% Working Loaners is more likely to default", 
         fontsize=20, fontweight='bold')
plt.text(x= -1, y=51, s="In the graphic image below, Some of loaner by job category is have\nPotential to default", 
         fontsize=18)

plt.xlabel('Job Category', fontsize=14)
plt.ylabel('(%) Total Loaners', fontsize=14)

## Education Type VS Target

In [ ]:
df['NAME_EDUCATION_TYPE'].value_counts()

In [ ]:
df_edu = df.groupby(['NAME_EDUCATION_TYPE','TARGET']).agg({'SK_ID_CURR' : ['nunique']})
df_edu = df_edu.reset_index()
df_edu.columns = ['EDUCATION CATEGORY','TARGET STATUS','TOTAL LOANERS']
df_edu['RATIO'] = (df_edu['TOTAL LOANERS'])/len(df)*100
df_edu['RATIO ROUNDED'] = df_edu['RATIO'].apply(lambda x: round(x, decimals))
df_edu

In [ ]:
sns.displot(x='NAME_EDUCATION_TYPE', hue='TARGET', data=df, stat = 'percent', height=5, aspect=2.5)

plt.text(x= -1, y=80, s="Around 6% Loaners in Secondary / secondary special more likely to default then others", 
         fontsize=20, fontweight='bold')
plt.text(x= -1, y=70, s="In the graphic image below, Some of loaner by edu category is have\nPotential to default", 
         fontsize=18)

plt.xlabel('Education Category', fontsize=14)
plt.ylabel('(%) Total Loaners', fontsize=14)

## Marital Status VS Target

In [ ]:
df['NAME_FAMILY_STATUS'].value_counts()

In [ ]:
df_marit = df.groupby(['NAME_FAMILY_STATUS','TARGET']).agg({'SK_ID_CURR' : ['nunique']})
df_marit = df_marit.reset_index()
df_marit.columns = ['MARITAL STATUS','TARGET STATUS','TOTAL LOANERS']
df_marit['RATIO'] = (df_marit['TOTAL LOANERS'])/len(df)*100
df_marit['RATIO ROUNDED'] = df_marit['RATIO'].apply(lambda x: round(x, decimals))
df_marit

In [ ]:
sns.displot(x='NAME_FAMILY_STATUS', hue='TARGET', data=df, stat = 'percent', height=5, aspect=2.5)

plt.text(x= -1, y=80, s="Around 5% Loaners in 'Married' is more likely to default then others", 
         fontsize=20, fontweight='bold')
plt.text(x= -1, y=70, s="In the graphic image below, Some of loaner by edu category is have\nPotential to default", 
         fontsize=18)

plt.xlabel('Marital Status', fontsize=14)
plt.ylabel('(%) Total Loaners', fontsize=14)

## House Type VS Target

In [ ]:
df['NAME_HOUSING_TYPE'].value_counts()

In [ ]:
df_house = df.groupby(['NAME_HOUSING_TYPE','TARGET']).agg({'SK_ID_CURR' : ['nunique']})
df_house = df_house.reset_index()
df_house.columns = ['HOUSE TYPE','TARGET STATUS','TOTAL LOANERS']
df_house['RATIO'] = (df_house['TOTAL LOANERS'])/len(df)*100
df_house['RATIO ROUNDED'] = df_house['RATIO'].apply(lambda x: round(x, decimals))
df_house

In [ ]:
sns.displot(x='NAME_HOUSING_TYPE', hue='TARGET', data=df, stat = 'percent', height=6, aspect=2)

plt.text(x= -1, y=103, s="Around 7% Loaner who has house is more likely to default then others", 
         fontsize=20, fontweight='bold')
plt.text(x= -1, y=91, s="In the graphic image below, Some of loaner who have house (7%) is have\nPotential to default", 
         fontsize=18)

plt.xlabel('House Type', fontsize=14)
plt.ylabel('(%) Total Loaners', fontsize=14)

## Gender VS Target

In [ ]:
df['CODE_GENDER'].value_counts()

In [ ]:
df_gender = df.groupby(['CODE_GENDER','TARGET']).agg({'SK_ID_CURR' : ['nunique']})
df_gender = df_gender.reset_index()
df_gender.columns = ['GENDER','TARGET STATUS','TOTAL LOANERS']
df_gender['RATIO'] = df_gender['TOTAL LOANERS']/len(df)*100
df_gender['RATIO ROUNDED'] = df_gender['RATIO'].apply(lambda x: round(x, decimals))
df_gender

In [ ]:
sns.displot(x='CODE_GENDER', hue='TARGET', data=df, stat = 'percent', height=6, aspect=2)

plt.text(x= -0.9, y=77, s="Female Loaners has 5% chance to defult", 
         fontsize=20, fontweight='bold')
plt.text(x= -0.9, y=68, s="14170 Female Loaners is have more Potential \nto default (5%) then the Male Loaners", 
         fontsize=18)

plt.xlabel('Gender', fontsize=14)
plt.ylabel('(%) Total Loaners', fontsize=14)

# Data Pre-processing

## Handling High Correlation

In [ ]:
corr_matrix = df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop_hicorr = [column for column in upper.columns if any(upper[column] > 0.7)]

In [ ]:
to_drop_hicorr

In [ ]:
df.drop(to_drop_hicorr, axis=1, inplace=True)

In [ ]:
df.info()

## Handling Missing Values

### Check Missing Values

In [ ]:
# print the name of columns with missing values (so I can copy paste :D)
null = df.isnull().sum() * 100 / df.shape[0]
null[null > 0].sort_values(ascending=False)

### Drop Null Values > 55%

In [ ]:
df.drop(columns=['COMMONAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'FONDKAPREMONT_MODE', 'YEARS_BUILD_AVG', 
             'OWN_CAR_AGE', 'LANDAREA_AVG', 'BASEMENTAREA_AVG', 'EXT_SOURCE_1'], inplace=True)

### Filling Null Values <= 55%

In [ ]:
null = df.isnull().sum() * 100 / df.shape[0]
null[null > 0].sort_values(ascending=False)

In [ ]:
nums = [key for key in dict(df.dtypes) if dict(df.dtypes)[key] in ['float64', 'int64']]
cats = [key for key in dict(df.dtypes) if dict(df.dtypes)[key] in ['object']]

In [ ]:
#create two DataFrames, one for each data type
data_numeric = df[nums]
data_categorical = pd.DataFrame(df[cats])


from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='median', verbose=0)
data_numeric = pd.DataFrame(imp.fit_transform(data_numeric), columns = data_numeric.columns) #only apply imputer to numeric columns

cimp = SimpleImputer(missing_values=np.nan, strategy='most_frequent', verbose=0)
data_categorical = pd.DataFrame(cimp.fit_transform(data_categorical), columns = data_categorical.columns) #only apply imputer to numeric columns


#you could do something like one-hot-encoding of data_categorical here

#join the two masked dataframes back together
df = pd.concat([data_numeric, data_categorical], axis = 1)

In [ ]:
df.isna().sum().sum()

# Feature Scalling & Transformation

## One Hot Encoding

In [ ]:
categorical = [col for col in df.select_dtypes(include='object').columns.tolist()]

In [ ]:
onehot = pd.get_dummies(df[categorical], drop_first=True)

In [ ]:
onehot.head()

## Standardization

In [ ]:
#numerical = [col for col in df.select_dtypes(exclude='object').columns.tolist()]

In [ ]:
numerical = [col for col in df.columns.tolist() if col not in categorical + ['TARGET']]

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
std = pd.DataFrame(ss.fit_transform(df[numerical]), columns=numerical)

In [ ]:
std.head()

## Transformed Data

In [ ]:
df_model = pd.concat([onehot, std, df[['TARGET']]], axis=1)

In [ ]:
df_model = df_model.drop(['SK_ID_CURR'], axis = 1)

# Modeling

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df_model.drop('TARGET', axis=1)
y = df_model['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X.shape, y.shape

## Training

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(max_depth=4)
rfc.fit(X_train, y_train)

In [ ]:
arr_feature_importances = rfc.feature_importances_
arr_feature_names = X_train.columns.values
    
df_feature_importance = pd.DataFrame(index=range(len(arr_feature_importances)), columns=['feature', 'importance'])
df_feature_importance['feature'] = arr_feature_names
df_feature_importance['importance'] = arr_feature_importances
df_all_features = df_feature_importance.sort_values(by='importance', ascending=False)
df_all_features

## Validation

In [ ]:
y_pred_proba = rfc.predict_proba(X_test)[:][:,1]

df_actual_predicted = pd.concat([pd.DataFrame(np.array(y_test), columns=['y_actual']), pd.DataFrame(y_pred_proba, columns=['y_pred_proba'])], axis=1)
df_actual_predicted.index = y_test.index

### AUC

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
fpr, tpr, tr = roc_curve(df_actual_predicted['y_actual'], df_actual_predicted['y_pred_proba'])
auc = roc_auc_score(df_actual_predicted['y_actual'], df_actual_predicted['y_pred_proba'])

plt.plot(fpr, tpr, label='AUC = %0.4f' %auc)
plt.plot(fpr, fpr, linestyle = '--', color='k')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()

### KS

In [ ]:
df_actual_predicted = df_actual_predicted.sort_values('y_pred_proba')
df_actual_predicted = df_actual_predicted.reset_index()

df_actual_predicted['Cumulative N Population'] = df_actual_predicted.index + 1
df_actual_predicted['Cumulative N Bad'] = df_actual_predicted['y_actual'].cumsum()
df_actual_predicted['Cumulative N Good'] = df_actual_predicted['Cumulative N Population'] - df_actual_predicted['Cumulative N Bad']
df_actual_predicted['Cumulative Perc Population'] = df_actual_predicted['Cumulative N Population'] / df_actual_predicted.shape[0]
df_actual_predicted['Cumulative Perc Bad'] = df_actual_predicted['Cumulative N Bad'] / df_actual_predicted['y_actual'].sum()
df_actual_predicted['Cumulative Perc Good'] = df_actual_predicted['Cumulative N Good'] / (df_actual_predicted.shape[0] - df_actual_predicted['y_actual'].sum())

In [ ]:
df_actual_predicted.head()

In [ ]:
KS = max(df_actual_predicted['Cumulative Perc Good'] - df_actual_predicted['Cumulative Perc Bad'])

plt.plot(df_actual_predicted['y_pred_proba'], df_actual_predicted['Cumulative Perc Bad'], color='r')
plt.plot(df_actual_predicted['y_pred_proba'], df_actual_predicted['Cumulative Perc Good'], color='b')
plt.xlabel('Estimated Probability for Being Bad')
plt.ylabel('Cumulative %')
plt.title('Kolmogorov-Smirnov:  %0.4f' %KS)

Model yang dibangun menghasilkan performa AUC = 0.7142 dan KS = 0.32. Pada dunia credit risk modeling, umumnya AUC di atas 0.7 dan KS di atas 0.3 sudah termasuk performa yang baik.